Note for exporting:

    Need to export to html.
    
    Can't then make the html into a pdf without plots not fitting

In [19]:
import os 
ROOT = os.getenv( "HOME" )
import pandas as pd
import numpy as np
pd.set_option('precision', 2)
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import seaborn as sns
sns.set(style="whitegrid")
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf')

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Band, Span, Plot, HoverTool
# output to static HTML 
output_notebook()

import datetime

pd.options.display.max_rows = 999
pd.options.plotting.matplotlib.register_converters = True

Loading BokehJS ...

In [2]:
PLACE = 'North Hollywood'
PLACES = [  'North Hollywood', 'Burbank', 'Encino', 'Glendale', 'Sherman Oaks',  'Studio City',   'Van Nuys',  'Valley Glen',]

# numbers are small or null before this
start_date = '2020-03-24'

county_url = 'https://raw.githubusercontent.com/datadesk/california-coronavirus-data/master/latimes-county-totals.csv'
locale_url = 'https://raw.githubusercontent.com/datadesk/california-coronavirus-data/master/latimes-place-totals.csv'
hospital_url = 'https://raw.githubusercontent.com/datadesk/california-coronavirus-data/master/cdph-hospital-patient-county-totals.csv'

# Download and prepare data

In [3]:
def calc_daily_new(locales, place, replace_neg=False):
    """
    Returns a dataframe with the change between each day
    """
    d = locales.loc[place].copy(deep=True)
    try:
        d.drop(['county'], axis=1, inplace=True)
    except:
        pass
    d = d - d.shift(1)
    if replace_neg:
        return d.clip(lower=0)
    return d

def calc_daily_growth(locales, place, prevent_negatives=True):
    """
    Calculates the daily growth rate
    
    If prevent_negatives is True, it will filter out any negative
    growth values (as sometimes happens when negative cases /deaths 
    have been added to the data to adjust totals) 
    """
    
    daily =  calc_daily_new(locales, place)
    d = daily / daily.shift(1) 
    
    # remove any infinite values 
    d = d.mask(np.isinf(d))
    
    # remove any negative values
    if prevent_negatives:
        d = d[d.confirmed_cases >= 0]
    
    return d


def plot_growth_rate(growth_frame, place):
    fig, axes = plt.subplots(figsize=(10, 4))
    title = "{} Daily growth rate".format(place)
    growth_frame.plot(kind='bar', title=title, ax=axes)
    ticklabels = [i.strftime("%b-%d") for i, row in growth_frame.iterrows()]
    axes.xaxis.set_major_formatter(mticker.FixedFormatter(ticklabels))
    # fig.autofmt_xdate();
    axes.axhline(y=1, color='r')
    fig.tight_layout()

    
# noho_growth_rate

In [4]:
counties = pd.read_csv(county_url)
counties.date = pd.to_datetime(counties.date)
counties.set_index('date', inplace=True)
counties.sort_index(inplace=True)
counties.drop(['fips'], axis=1,inplace=True)

# LA County data
lac = counties[counties.county == 'Los Angeles'].loc[ start_date : ].copy(deep=True)
lac.drop(['county'], axis=1, inplace=True)
lac_daily_new = lac[['new_confirmed_cases', 'new_deaths']]
lac_cum = lac[[ 'confirmed_cases', 'deaths']]
lac_growth_rate = lac_daily_new / lac_daily_new.shift(1)

# Locale data
locales = pd.read_csv(locale_url)
locales.drop([  'fips',  'note', 'x', 'y'], axis=1, inplace=True)
locales.date = pd.to_datetime(locales.date)
locales.set_index(['place', 'date'], inplace=True)
locales.sort_index(inplace=True)
la_locales = locales[locales.county == 'Los Angeles'].copy(deep=True)
la_locales.drop([ 'county'], axis=1, inplace=True)
la_locale_names = list(la_locales.index.get_level_values(0))

# NoHo data
noho = la_locales.loc['North Hollywood'].copy(deep=True)

# hospitalizations 
hosp = pd.read_csv(hospital_url)
hosp.date = pd.to_datetime(hosp.date)
hosp.drop(['fips'], axis=1, inplace=True)
hosp_lac = hosp[hosp.county == 'Los Angeles'].drop(['county'], axis=1).copy(deep=True)
hosp_lac.set_index('date', inplace=True); hosp_lac.sort_index(inplace=True)
hosp_lac['total_patients'] = hosp_lac.positive_patients + hosp_lac.suspected_patients
hosp_lac['total_icu'] = hosp_lac.icu_positive_patients + hosp_lac.icu_suspected_patients
new_hosp_lac = hosp_lac - hosp_lac.shift(1)
new_hosp_lac.dropna(inplace=True)

# growth rates
# dict version for easiest access to individual frames
locale_growth_rates = { place : calc_daily_growth(locales, place) for place in PLACES}
# combined version 
fms = []
for place, frame in locale_growth_rates.items():
    f = frame.copy(deep=True)
    f['locale'] = place
    fms.append(f)
growth_rate_frame = pd.concat(fms)

noho_growth_rate = locale_growth_rates.get(PLACE)
noho_growth_rate = noho_growth_rate[noho_growth_rate.confirmed_cases >= 0]

# Daily new

In [22]:
def plot_daily(frame, field_name, title_word, place='(LAC)', trim_pos=None, trim_neg=None):
    """
    Plots the desired field's daily values as a vertical bar graph
    with a 7-day moving average line
    
    trim_pos: None or float. If set, will remove values above this limit from the bars (but not average)
    trim_neg: None or float. If set, will remove values below this limit from the bars (but not average)
    
    """
    TOOLTIPS = [
        ("cases", f"@{field_name}"),
        ("date", "@date")
    ]

    p = figure(
        plot_width=1000,
        plot_height=400,
        title=f"Daily new {title_word} {place}",
        x_axis_type='datetime') #,
#                tooltips=TOOLTIPS)


    hover = HoverTool(tooltips= TOOLTIPS, #[('date', '@DateTime{%F}')],
          formatters={'date': 'datetime'})
    
    p.add_tools(hover)

    bar_data = frame.reset_index()
    if trim_pos is not None:
        bar_data = bar_data[bar_data[field_name] <= trim_pos]
    if trim_neg is not None:
        bar_data = bar_data[bar_data[field_name] >= trim_neg]
        
    source = ColumnDataSource(bar_data)
    source2 = ColumnDataSource(frame.rolling(window='7D').mean().reset_index())

    # add a circle renderer with a size, color, and alpha
    # p.circle(x='date', y='new_confirmed_cases', size=10, color="navy", source=source, alpha=0.5)
    p.vbar(
        x='date',
        top=field_name,
        color="green",
        width=0.4,
        source=source,
        alpha=0.8,
        legend_label=f'Daily new {title_word}')
    p.line(
        x='date',
        y=field_name,
        line_width=5,
        color="navy",
        source=source2,
        alpha=0.3,
        legend_label='7-day rolling avg')
 
    # df['lower'] = df.new_confirmed_cases.mean() - df.new_confirmed_cases.std()
    # df['upper'] = df.new_confirmed_cases.mean() + df.new_confirmed_cases.std()
    # band = Band(base='x', lower='lower', upper='upper', source=source, level='underlay',
    #             fill_alpha=1.0, line_width=1, line_color='black')
    # p.add_layout(band)

    # show the results
    show(p)

## LA County

In [23]:
cases_d = {
    'frame': lac_daily_new,
    'field_name': 'new_confirmed_cases',
    'title_word': 'cases',
    'place': '-- LA County'
}

deaths_d = {
    'frame': lac_daily_new,
    'field_name': 'new_deaths',
    'title_word': 'deaths',
    'place': '-- LA County'
}

plot_daily(**cases_d)
plot_daily(**deaths_d)

The above represents the daily new case  and death counts for LA County. It would be nice if these lines were sloping downward....

Given that it takes 2-3 weeks for a patient to progress from infection to death, comparing the two plots seems interesting. (Whether this is actually interesting is above my paygrade.)  They seem to track together rather than being offset by 2-3 weeks. Though the sharp drop in new cases at the end of April does seem to have approximately this offset from the sharp drop in deaths toward late May. 

With the sharp rise in cases at the very end of May/ beginning of June we should expect to see a rise in deaths around the end of June.

Notes

    The spike in late April is when testing finally ramped up.

    The data has some day-of-week seasonality for both cases and deaths. The low numbers generally occur on Sundays and Mondays. I've taken a 7 day rolling average to smooth those out. 

In [7]:
pos_patients_d = {
    'frame': new_hosp_lac,
    'field_name': 'total_patients',
    'title_word': 'hospitalizations (suspected and confirmed cases)',
    'place': '-- LA County',
    'trim_pos': 100,
    'trim_neg': -100
}
pos_icu_d = {
    'frame': new_hosp_lac,
    'field_name': 'total_icu',
    'title_word': 'patients in ICU (suspected and confirmed cases)',
    'place': '-- LA County',
    'trim_pos': 40,
    'trim_neg': -40
}
plot_daily(**pos_patients_d)
plot_daily(**pos_icu_d)

The above charts represent the difference between the number of hospitalized covid19 patients on a given day and the count of the day before.

Thus starting around May 1st the number of hospitalized patients tended to decline each day; in early June the numbers begin to creep back up.

Notes

    - These plots are trimmed to remove plots of a few large outliers. Many of those outliers seem to be data adjustments since a similar large negative number occurs a few days after a large positive outlier  (e.g., +200 and -200). I have left these values in the calculation of the trendlines. 

    - A negative number on a given day means that more patients died / were discharged than were newly admitted.

    - The dataset contains separate fields for patients with positive tests and suspected patients. After the early days, the counts of suspected patients (both hospitalized and in icu) dropped off as testing got better. Thus I've assumed that most suspected patients were in fact infected and combined the counts of suspected and confirmed cases. 

In [8]:
TOOLTIPS = [
    ("total", "@total_patients"),
#         ("date", "@date")
]

p = figure(
    plot_width=1000,
    plot_height=400,
    title=f"Hospitalized patients ",
    x_axis_type='datetime',
           tooltips=TOOLTIPS)

#     bar_data = frame.reset_index()
#     if trim_pos is not None:
#         bar_data = bar_data[bar_data[field_name] <= trim_pos]
#     if trim_neg is not None:
#         bar_data = bar_data[bar_data[field_name] >= trim_neg]
        
source = ColumnDataSource(hosp_lac.reset_index())
source2 = ColumnDataSource(hosp_lac.rolling(window='7D').mean().reset_index())

# add a circle renderer with a size, color, and alpha
# p.circle(x='date', y='new_confirmed_cases', size=10, color="navy", source=source, alpha=0.5)
p.vbar(
    x='date',
    top='total_patients',
    color="green",
    width=0.4,
    source=source,
    alpha=0.8,
    legend_label='Total patients')
p.line(
    x='date',
    y='total_patients',
    line_width=5,
    color="navy",
    source=source2,
    alpha=0.3,
    legend_label='7-day rolling avg')

show(p)

TOOLTIPS = [
    ("total", "@total_icu"),
#         ("date", "@date")
]

p = figure(
    plot_width=1000,
    plot_height=400,
    title=f"ICU patients ",
    x_axis_type='datetime',
           tooltips=TOOLTIPS)


source = ColumnDataSource(hosp_lac.reset_index())
source2 = ColumnDataSource(hosp_lac.rolling(window='7D').mean().reset_index())

# add a circle renderer with a size, color, and alpha
# p.circle(x='date', y='new_confirmed_cases', size=10, color="navy", source=source, alpha=0.5)
p.vbar(
    x='date',
    top='total_icu',
    color="green",
    width=0.4,
    source=source,
    alpha=0.8,
    legend_label='Total patients')
p.line(
    x='date',
    y='total_icu',
    line_width=5,
    color="navy",
    source=source2,
    alpha=0.3,
    legend_label='7-day rolling avg')

show(p)

## NoHo

In [9]:
noho_new = calc_daily_new(locales, PLACE)
cases_d = {
    'frame': noho_new,
    'field_name': 'confirmed_cases',
    'title_word': 'cases',
    'place': '-- NoHo'
}

plot_daily(**cases_d)

The above represents the daily new cases in North Hollywood. The numbers are small and noisy.

Notes

    The spike in late April is when testing finally ramped up.

    The data has some day-of-week seasonality for both cases. The low numbers generally occur on Sundays and Mondays. I've taken a 7 day rolling average to smooth those out. 
    
    The day with negative new cases in early April is likely due to a correction to the totals. Either that or time temporarily and locally ran backwards.

## Neighboring locales

The following are daily new case counts for other south SF Valley neighborhoods.

I've included the North Hollywood case counts (from the above chart) for reference.

In [10]:
def plot_daily_locales(localities_frame, place, noho_new=None):
    """
    Plots the new case counts for localities as a vertical bar graph
    with a 7-day moving average line and a representation of north hollywood's trend 
    
    """
    field_name = 'confirmed_cases'

    p = figure(
        plot_width=1000,
        plot_height=400,
        title=f"Daily new cases --{place}",
        x_axis_type='datetime')

    d_new = calc_daily_new(localities_frame, place, replace_neg=True)
    source = ColumnDataSource(d_new.reset_index())
    source2 = ColumnDataSource(d_new.rolling(window='7D').mean().reset_index())

    p.vbar(
        x='date',
        top=field_name,
        color="green",
        width=0.4,
        source=source,
        alpha=0.8,
        legend_label=f'Daily new cases')
    p.line(
        x='date',
        y=field_name,
        line_width=5,
        color="navy",
        source=source2,
        alpha=0.3,
        legend_label=f'7-day rolling avg ({place})')

    if noho_new is not None:
        source3 = ColumnDataSource(
            noho_new.rolling(window='7D').mean().reset_index())
        p.line(
            x='date',
            y=field_name,
            line_width=5,
            color="red",
            source=source3,
            alpha=0.2,
            legend_label=f'7-day rolling avg (NoHo)')

    show(p)

In [11]:
for place in PLACES:
    if place != PLACE:
        plot_daily_locales(locales, place, noho_new=noho_new)

# Cumulative counts

## Cases (LA County)

Putting everything on a log scale allows us to see whether things are accelerating. The more diagonal the line, the more exponential the growth (i.e., a bigger exponent).

In [12]:
source = ColumnDataSource(lac)
p = figure(plot_width=1000, plot_height=400, x_axis_type='datetime',
           y_axis_type='log',
           title=f"Cumulative cases (log scale)", 
           title_location='above')

p.circle(x='date', 
         y='confirmed_cases', 
         size=5, 
         color="navy", 
         source=source, 
         alpha=0.2, 
         legend_label='Cases')

p.line(x='date', 
         y='confirmed_cases', 
         line_width=5, 
         color="navy", 
         source=source, 
         alpha=0.5, 
         legend_label='Cases')

p.legend.location = 'bottom_right'

show(p)

## Deaths (LA County)

In [13]:
source = ColumnDataSource(lac)
p = figure(plot_width=1000, plot_height=400, x_axis_type='datetime',
           y_axis_type='log',
           title=f"Cumulative deaths (log scale)", 
           title_location='above')

p.circle(x='date', 
         y='deaths', 
         size=5, 
         color="navy", 
         source=source, 
         alpha=0.2, 
         legend_label='Deaths')

p.line(x='date', 
         y='deaths', 
         line_width=5, 
         color="navy", 
         source=source, 
         alpha=0.5, 
         legend_label='Deaths')

p.legend.location = 'bottom_right'

show(p)

# Growth rates

We also can look at the growth rate from day-to-day. This is the number of new cases today divided by the number of new cases yesterday. For example:

    Yesterday: 10 new
    
    Today: 5 new
    
    Growth rate: 5 / 10 = 0.5
    
and

    Yesterday: 5 new
    
    Today: 10 new
    
    Growth rate: 2.0

If the growth rate is greater than 1, things are getting increasingly worse. If it is 1, they are staying the same. If it is less than 1, it is slowing. Thus in the following plots we want to see values below the red line. 



I've labeled the following plots 'trimmed' to indicate that outlier daily growth values about 5.0 are not represented by points. It's easy to get that kind of noise given the relatively small numbers in the local case data. The outlier values are included in the calculation of the trend lies.

---

NB, the rate cannot go below 0 because there can't be negative new cases. Sometimes the raw data contains negative values to make a correction to the totals. Since they comprise a very small number of cases, I've dropped all negative values.

## Deaths growth rate

In [14]:
# need to trim so plot is readable
top_limit=5; bottom_limit=-5
trm = lac_growth_rate[lac_growth_rate.new_deaths >= bottom_limit] # and noho_growth_rate.confirmed_cases <= top_limit]
trm = trm[trm.new_deaths <= top_limit]
source = ColumnDataSource(trm.reset_index())
# not using the trimmed source to compute trend
source2 = ColumnDataSource(lac_growth_rate.rolling(window='7D').mean().reset_index())

p = figure(plot_width=1000, plot_height=400, x_axis_type='datetime', 
           title=f"Daily death growth rate (trimmed)", 
           title_location='above')

p.circle(x='date', 
         y='new_deaths', 
         size=5, 
         color="navy", 
         source=source, 
         alpha=0.8, 
         legend_label='daily death growth rate')

p.line(
    x='date',
    y='new_deaths',
    line_width=5,
    color="navy",
    source=source2,
    alpha=0.5,
    legend_label=f'7-day rolling avg')

ln = Span(location=1, dimension='width', line_color='red', line_dash='dashed', line_width=3)
p.add_layout(ln)
p.legend.location = 'bottom_right'
show(p)

## Local growth rates

The following represent the growth rates of cases in various areas. To get a sense of whether an area is growing faster or slower than other areas I have overlaid the LA County and North Hollywood growth rates. 

NB, these plots shouldn't be taken terribly seriously. The number of cases in each area is relatively small. Thus it's easy to get outsized effects (e.g., if 1 case was diagnosed yesterday and 5 were diagnosed today, the growth rate would be 5). Still, taken as a whole they may be somewhat useful.

In [15]:
def plot_locale_growth(growth_frame, place, noho_growth_rate=None, lac_growth_rate=None, top_limit=5, bottom_limit=0):
    """
    Plots the growth rates for localities as a dot plot 
    with a 7-day moving average line and a representation of north hollywood's trend 
    
    """
    # trim the values so the plot will be readable
    trm = growth_frame[growth_frame.confirmed_cases >= bottom_limit] # and noho_growth_rate.confirmed_cases <= top_limit]
    trm = trm[trm.confirmed_cases <= top_limit]
    source = ColumnDataSource(trm.reset_index())
    # compute trend on the real data
    source2 = ColumnDataSource(growth_frame.rolling(window='7D').mean().reset_index())

    p = figure(plot_width=1000, plot_height=400, x_axis_type='datetime', 
               title=f"{place} case growth rate (trimmed)", 
               title_location='above')
    
    p.circle(x='date', 
             y='confirmed_cases', 
             size=5, 
             color="navy", 
             source=source, 
             alpha=0.8, 
             legend_label='daily case growth rate')

    p.line(
        x='date',
        y='confirmed_cases',
        line_width=5,
        color="navy",
        source=source2,
        alpha=0.5,
        legend_label=f'7-day rolling avg ({place})')
    
    if noho_growth_rate is not None and place != 'North Hollywood':
        source3 = ColumnDataSource(noho_growth_rate.rolling(window='7D').mean().reset_index())
        p.line(
            x='date',
            y='confirmed_cases',
            line_width=5,
            color="red",
            source=source3,
            alpha=0.2,
            legend_label=f'7-day rolling avg (NoHo)')
   
    if lac_growth_rate is not None:
        source4 = ColumnDataSource(lac_growth_rate.rolling(window='7D').mean().reset_index())
        p.line(
            x='date',
            y='new_confirmed_cases',
            line_width=5,
            color="grey",
            source=source4,
            alpha=0.2,
            legend_label=f'7-day rolling avg (LA County)')

    ln = Span(location=1, dimension='width', line_color='red', line_dash='dashed', line_width=3)
    p.add_layout(ln)
    p.legend.location = 'bottom_left'
#     show the results
    show(p)

In [16]:
for place, growth_frame in locale_growth_rates.items():
    plot_locale_growth(growth_frame, place, noho_growth_rate=noho_growth_rate, lac_growth_rate=lac_growth_rate)